## Load libraries

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import collections
import random
from scipy import spatial

fable_text = """
long ago , the mice had a general council to consider what measures
they could take to outwit their common enemy , the cat . some said
this , and some said that but at last a young mouse got up and said
he had a proposal to make , which he thought would meet the case . 
you will all agree , said he , that our chief danger consists in the
sly and treacherous manner in which the enemy approaches us . now , 
if we could receive some signal of her approach , we could easily
escape from her . i venture , therefore , to propose that a small
bell be procured , and attached by a ribbon round the neck of the cat
. by this means we should always know when she was about , and could
easily retire while she was in the neighbourhood . this proposal met
with general applause , until an old mouse got up and said that is
all very well , but who is to bell the cat ? the mice looked at one
another and nobody spoke . then the old mouse said it is easy to
propose impossible remedies .
"""
# Replace the carriage return with space.
fable_text = fable_text.replace('\n','')

#this function puts all the words in a single column vector within a numpy array

def read_data(raw_text):
 content = raw_text
 content = content.split() #splits the text by spaces (default split character)
 content = np.array(content)
 content = np.reshape(content, [-1, ])
 return content

training_data = read_data(fable_text)

dictionary = np.load('dictionary.npy').item()
reverse_dictionary = np.load('reverse_dictionary.npy').item()


### Restore graph and variables

In [2]:
saver = tf.train.import_meta_graph('RNN_model-500.meta')
sess= tf.Session()
saver.restore(sess,tf.train.latest_checkpoint('./'))
x,W,pred = tf.get_collection('ops')

embedding = sess.run(W)
tree = spatial.KDTree(embedding,5)

offset = 0
n_input = 3 
end_offset = n_input+1

while offset <= (len(training_data) - end_offset):
     x_integers = [[dictionary[str(training_data[i])]] for i in range(offset, offset+n_input)]
     x_integers = np.reshape(np.array(x_integers), [-1, n_input])
 
     # create embedding for target vector 
 
     y_position = offset+n_input

     pred_ = sess.run(pred, feed_dict = {x: x_integers})
 
     words_in = [str(training_data[i]) for i in range(offset, offset+n_input)] 
     target_word = str(training_data[y_position])
     nearest_dist,nearest_idx = tree.query(pred_[0],2)
     nearest_words = [reverse_dictionary[idx] for idx in nearest_idx]
     print("%s - [%s] vs [%s]" % (words_in, target_word, nearest_words))
     offset += 1 
sess.close()

INFO:tensorflow:Restoring parameters from ./RNN_model-500
['long', 'ago', ','] - [the] vs [['the', 'which']]
['ago', ',', 'the'] - [mice] vs [['cat.', 'easilyescape']]
[',', 'the', 'mice'] - [had] vs [['had', 'but']]
['the', 'mice', 'had'] - [a] vs [['a', 'this']]
['mice', 'had', 'a'] - [general] vs [[',', '.']]
['had', 'a', 'general'] - [council] vs [['council', '.']]
['a', 'general', 'council'] - [to] vs [['to', 'would']]
['general', 'council', 'to'] - [consider] vs [['consider', 'should']]
['council', 'to', 'consider'] - [what] vs [['what', 'that']]
['to', 'consider', 'what'] - [measuresthey] vs [['measuresthey', 'saidthis']]
['consider', 'what', 'measuresthey'] - [could] vs [['could', 'would']]
['what', 'measuresthey', 'could'] - [take] vs [['take', '.']]
['measuresthey', 'could', 'take'] - [to] vs [['to', 'take']]
['could', 'take', 'to'] - [outwit] vs [['easilyescape', 'smallbell']]
['take', 'to', 'outwit'] - [their] vs [['their', 'and']]
['to', 'outwit', 'their'] - [common] vs [[

[',', 'until', 'an'] - [old] vs [['old', ',']]
['until', 'an', 'old'] - [mouse] vs [['mouse', 'mice']]
['an', 'old', 'mouse'] - [got] vs [['got', 'but']]
['old', 'mouse', 'got'] - [up] vs [['up', ',']]
['mouse', 'got', 'up'] - [and] vs [['and', ',']]
['got', 'up', 'and'] - [said] vs [['saidhe', ',']]
['up', 'and', 'said'] - [that] vs [['that', 'but']]
['and', 'said', 'that'] - [isall] vs [['but', '.']]
['said', 'that', 'isall'] - [very] vs [['very', 'but']]
['that', 'isall', 'very'] - [well] vs [['well', ',']]
['isall', 'very', 'well'] - [,] vs [[',', '.']]
['very', 'well', ','] - [but] vs [['but', 'that']]
['well', ',', 'but'] - [who] vs [[',', '.']]
[',', 'but', 'who'] - [is] vs [['is', '.']]
['but', 'who', 'is'] - [to] vs [['to', 'but']]
['who', 'is', 'to'] - [bell] vs [['oneanother', 'couldeasily']]
['is', 'to', 'bell'] - [the] vs [['the', 'which']]
['to', 'bell', 'the'] - [cat] vs [['cat', 'easilyescape']]
['bell', 'the', 'cat'] - [?] vs [['.', '?']]
['the', 'cat', '?'] - [the] vs